In [1]:
import pandas as pd
import sqlite3
from scipy.stats import chi2_contingency

# Dữ liệu gốc
data = {
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
}
df_wide = pd.DataFrame(data)

# Chuyển về dạng quan hệ (long format)
df_long = df_wide.melt(id_vars=['Day'], var_name='Model', value_name='Score')

# Chuyển thành bảng trong SQLite
conn = sqlite3.connect(":memory:")
df_long.to_sql("scores", conn, index=False, if_exists="replace")

# Truy vấn SQL: tính tần suất điểm (đã làm tròn) theo mẫu xe
query = """
SELECT 
    Model,
    ROUND(Score) AS RoundedScore,
    COUNT(*) AS Frequency
FROM scores
GROUP BY Model, RoundedScore
ORDER BY Model, RoundedScore;
"""

result = pd.read_sql_query(query, conn)
print(" Bảng tần suất điểm đã làm tròn theo mẫu xe:")
print(result)


 Bảng tần suất điểm đã làm tròn theo mẫu xe:
  Model  RoundedScore  Frequency
0     A           6.0          1
1     A           7.0          1
2     A           8.0          2
3     B           6.0          1
4     B           7.0          1
5     B           9.0          2
6     C           5.0          1
7     C           7.0          2
8     C           8.0          1


In [2]:
# Tạo bảng chéo từ dữ liệu đã group
pivot = pd.pivot_table(df_long, index="Model", columns="Day", values="Score", aggfunc="mean")

# Hoặc nếu muốn kiểm định phân phối điểm giữa các mẫu:
contingency = pd.crosstab(df_long['Model'], round(df_long['Score']))

# Thực hiện kiểm định chi-squared
chi2, p, dof, expected = chi2_contingency(contingency)

print("\n Chi-squared kiểm định giữa Model và điểm số:")
print(f"Chi² = {chi2:.4f}, p-value = {p:.4f}")



 Chi-squared kiểm định giữa Model và điểm số:
Chi² = 6.0000, p-value = 0.6472


In [3]:
import pandas as pd
import sqlite3

# Tạo dataframe
data = [
    ("Day 1", "A", 8),
    ("Day 1", "B", 9),
    ("Day 1", "C", 7),
    ("Day 2", "A", 7.5),
    ("Day 2", "B", 8.5),
    ("Day 2", "C", 7),
    ("Day 3", "A", 6),
    ("Day 3", "B", 7),
    ("Day 3", "C", 8),
    ("Day 4", "A", 7),
    ("Day 4", "B", 6),
    ("Day 4", "C", 5),
]

df = pd.DataFrame(data, columns=["Day", "Model", "Score"])

# Làm tròn điểm số để phù hợp với kiểm định χ²
df["RoundedScore"] = df["Score"].round()

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
df.to_sql("scores", conn, index=False, if_exists="replace")


12

In [4]:
query = """
SELECT Model, RoundedScore, COUNT(*) as Frequency
FROM scores
GROUP BY Model, RoundedScore
ORDER BY Model, RoundedScore;
"""

freq_df = pd.read_sql_query(query, conn)
print(freq_df)


  Model  RoundedScore  Frequency
0     A           6.0          1
1     A           7.0          1
2     A           8.0          2
3     B           6.0          1
4     B           7.0          1
5     B           8.0          1
6     B           9.0          1
7     C           5.0          1
8     C           7.0          2
9     C           8.0          1


In [5]:
from scipy.stats import chi2_contingency

# Tạo bảng chéo (contingency table)
pivot = pd.pivot_table(df, values="Score", index="Model", columns="RoundedScore", aggfunc='count', fill_value=0)

# Kiểm định χ²
chi2, p, dof, expected = chi2_contingency(pivot)

print("Chi-squared =", round(chi2, 4))
print("p-value =", round(p, 4))


Chi-squared = 6.0
p-value = 0.6472


In [10]:
print('Từ kết quả ta thấy p-value = 0.6472 > 0.05 ==> không có sự khác biệt rõ ràng trong 4 ngày ')

print('và kết quả thử nghiệm phụ thuộc vào mẫu xe')

Từ kết quả ta thấy p-value = 0.6472 > 0.05 ==> không có sự khác biệt rõ ràng trong 4 ngày 
và kết quả thử nghiệm phụ thuộc vào mẫu xe
